# Process all lmbd datasets to Pandas Dataframes

Information currently stored in Dataframes: 

- Adsorbate: formula, interneal index
- Bulk: SLICES, Materials Project database identifier (mpid), formula, internal index
- Surface: *hkl* Miller indices
- Target: Adsorption energy in eV
- metadata: data class (int), anomaly class(int)

In [1]:
import os

import pandas as pd
from fairchem.core.datasets import LmdbDataset

## Process all .lmdb files and generate df

Keep the same directory structure as the folder with the .lmdb

In [2]:
SPLIT = '10k'  # 10k, 100k, or all
ROOT_DF = f'../data/is2res_train_val_test_dfs/{SPLIT}'
ROOT_LMDB = f'../data/is2res_train_val_test_lmdbs/data/is2re/{SPLIT}'
OC20_INFO = "../data/oc20_data_mapping.pkl"
ADSORBATE_DB = "../data/adsorbate_db_2021apr28.pkl"
SLICES_MAP = "../data/bulk/mapping_mpid_to_slice.pkl"
ADSORBATE_DICT = "../data/adsorbate_string_dict.pkl"

In [3]:
adsorbate_db = pd.read_pickle(ADSORBATE_DB)
oc20_map = pd.read_pickle(OC20_INFO)
slices_map = pd.read_pickle(SLICES_MAP)
adsorbate_dict = pd.read_pickle(ADSORBATE_DICT)

In [4]:
def get_energy_adsorbate(c=0, h=0, n=0, o=0):
    Ec, Eh, En, Eo = -7.282, -3.477, -8.083, -7.204
    return c * Ec + h * Eh + n * En + o * Eo

In [ ]:
for dataset in os.listdir(ROOT_DF):
    print(dataset)
    lmdb_path = f'{ROOT_LMDB}/{dataset}/data.lmdb'
    df_path = f'{ROOT_DF}/{dataset}/data.parquet'
    db = LmdbDataset({'src': lmdb_path})

    print(f'Loaded {lmdb_path}')

    # adsorbate features
    ads_symbolss, ads_ids = [], []
    nC, nH, nO, nN = [], [], [], []
    ads_sizes = []
    ads_smiless, ads_inchis, ads_inchikeys = [], [], []
    ads_ase_formulas = []
    ads_energies_eV = []
    # material features
    bulk_symbolss, bulk_mpids, bulk_ids = [], [], []
    slicess = []
    natoms_bulk = []  # Number of atoms in the "unit" cell
    nelems_bulk = []  # Number of distinct elements (A, AB, ABC material)
    Cbulk, Hbulk, Obulk, Nbulk = [], [], [], []
    # surface features
    hs, ks, ls, hkls = [], [], [], []
    # data features
    sids, anomalies, classes, eads_eV = [], [], [], []
    scaled_eads_eV = []

    for data in db:
        key = f'random{data.sid}'
        info = oc20_map[key]
        sids.append(data.sid)
        # Adsorbate atom count
        C, H, O, N = 0, 0, 0, 0 
        for i in range(data.num_nodes):
            if data.tags[i] == 2:
                Z = data.atomic_numbers[i]
                if Z == 6:  # C
                    C += 1
                elif Z == 1:  # H
                    H += 1
                elif Z == 7:  # N
                    N += 1
                elif Z == 8:  # O
                    O += 1
                else:
                    raise Exception("Something is wronggg.")
        nC.append(C)
        nH.append(H)
        nO.append(O)
        nN.append(N)
        ads_sizes.append(C + H + O + N)
        anomalies.append(info['anomaly'])
        classes.append(info['class'])
        h, k, l = info['miller_index']
        hs.append(h)
        ks.append(k)
        ls.append(l)
        hkls.append(str(h) + str(k) + str(l))
        bulk_symbolss.append(info['bulk_symbols'])
        bulk_mpids.append(info['bulk_mpid'])
        bulk_ids.append(info['bulk_id'])
        slicess.append(slices_map[info['bulk_mpid']])
        ads_energies_eV.append(get_energy_adsorbate(C, H, N, O))
        ads_symbolss.append(info['ads_symbols'])
        ads_ids.append(info['ads_id'])
        if 'test' in dataset:  # Target unavailable
            eads_eV.append('N/A')
            scaled_eads_eV.append('N/A')
        else:
            eads_eV.append(data.y_relaxed)
            scaled_eads_eV.append(eads_eV[-1] + ads_energies_eV[-1])
        counter = 0
        elems = set()
        for i in slicess[-1].split(" "):
            if i.isnumeric():
                break
            counter += 1
            elems.add(i)
        nelems_bulk.append(len(elems))
        natoms_bulk.append(counter)
        ads_smiless.append(adsorbate_dict[info['ads_id']]["smiles"])
        ads_inchis.append(adsorbate_dict[info['ads_id']]["inchi"])
        ads_inchikeys.append(adsorbate_dict[info['ads_id']]["inchikey"])
        ads_ase_formulas.append(adsorbate_dict[info['ads_id']]["ase_formula"])
        Cbulk.append('C' in slicess[-1].split())
        Hbulk.append('H' in slicess[-1].split())
        Nbulk.append('N' in slicess[-1].split())
        Obulk.append('O' in slicess[-1].split())
        

    print(len(sids), len(natoms_bulk), len(nelems_bulk))

    df = pd.DataFrame({'sid': sids,
                  'anomaly': anomalies, 
                  'class': classes, 
                  'C': nC, 
                  'H': nH, 
                  'O': nO, 
                  'N': nN, 
                  'ads_size': ads_sizes, 
                  'ads_symbols': ads_symbolss, 
                  'ads_id': ads_ids, 
                  'ads_smiles': ads_smiless, 
                  'ads_inchi': ads_inchis, 
                  'ads_inchikeys': ads_inchikeys,
                  'ads_ase_formula': ads_ase_formulas,
                  'ads_energy_eV': ads_energies_eV, 
                  'bulk_symbols': bulk_symbolss, 
                  'bulk_mpid': bulk_mpids, 
                  'bulk_id': bulk_ids, 
                  'bulk_nelems': nelems_bulk, 
                  'bulk_natoms': natoms_bulk,
                  'C_in_bulk': Cbulk, 
                  'H_in_bulk': Hbulk,
                  'O_in_bulk': Obulk,
                  'N_in_bulk': Nbulk,
                  'slices': slicess,
                  'h': hs, 
                  'k': ks, 
                  'l': ls, 
                  'hkl': hkls, 
                  'eads_eV': eads_eV, 
                  'scaled_eads_eV': scaled_eads_eV})
    
    df.to_parquet(df_path)
    print(f'Saved {df_path}')    

In [4]:
df = pd.read_parquet(df_path)

# Add HetSMILES (order1)

For both initial state and final state, together with the 'full representation' boolean variable. This variable is True when the SMILES/graph includes all elements of the material, False if not. It could happen for cases where the material consists of multiple elements and the adsorbate is more prone to interact only with a subset of material elements.

In [9]:
def lines2lists(xx: list):
    idxs, fris, frfs, his, hfs = [], [], [], [], []
    for line in xx:
        y = line.replace("\n", "").split(" ")
        idxs.append(y[0])
        fris.append(y[1])
        frfs.append(y[3])
        his.append(y[2])
        hfs.append(y[4])
    return idxs, fris, his, frfs, hfs

In [ ]:
SPLIT = 'all'  # 10k, 100k, or all
SET = 'val_ood_both'
ORDER = 'o1'  # o1 or o2
DF_PATH = f'../data/is2res_train_val_test_dfs/{SPLIT}/{SET}/data.parquet'
# SMILES_PATH = f'../data/hetsmiles/{SPLIT}_{ORDER}.txt'
SMILES_PATH = f'../data/hetsmiles/both_{ORDER}.txt'
df = pd.read_parquet(DF_PATH)
df.head()

with open(SMILES_PATH, 'r') as f:
    lines = f.readlines()

print(lines[0])
print(len(lines), len(df))

In [ ]:
idxs, fris, his, frfs, hfs = lines2lists(lines)

len_his = [len(x) for x in his]
len_hfs = [len(x) for x in hfs]

print(len(len_his))

In [37]:
df['hetsmiles_IS_o1'] = his
df['hetsmiles_FS_o1'] = hfs
df['frh_IS_o1'] = fris
df['frh_FS_o1'] = frfs
df['len_his_o1'] = len_his
df['len_hfs_o1'] = len_hfs

In [39]:
df.to_parquet(DF_PATH)

## Generate adsorbate SMILES/InChI/InChIKey

- Should work for both closed-shell and open-shell molecules (CH4, CH3, CH2, CH3O, ...)

In [21]:
# adsorbate_dict = {}

# for idx, v in adsorbate_db.items():
#     y_dict = {}
#     v[0]._pbc = np.array([1,1,1])
#     atoms = v[0]
#     ase.io.write('test.pdb', atoms)
#     mol = Chem.rdmolfiles.MolFromPDBFile('test.pdb', sanitize=False, removeHs=True)
#     num_bonds = mol.GetNumBonds()
#     # https://github.com/IUPAC-InChI/InChI/blob/main/INCHI-1-DOC/InChI_UserGuide.pdf for options
#     inchi = Chem.MolToInchi(mol, options='/DoNotAddH')
#     inchikey = Chem.MolToInchiKey(mol, options='/DoNotAddH')
#     smiles = Chem.MolToSmiles(mol, allHsExplicit=False)
#     ase_formula = atoms.get_chemical_formula()
#     if smiles == '[HH]':
#         smiles = '[H]'
#     y_dict['smiles'] = smiles
#     y_dict['inchi'] = inchi    
#     y_dict['inchikey'] = inchikey
#     y_dict['ase_formula'] = ase_formula
#     adsorbate_dict[idx] = y_dict

# import pickle

# with open('adsorbate_string_dict.pkl', 'wb') as fp:
#     pickle.dump(adsorbate_dict, fp, protocol=pickle.HIGHEST_PROTOCOL)